In [11]:
import tweepy
import json
import time
from pandas import Series, DataFrame
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


# Authorization Codes
consumer_key = "ZCVjbgfaSGlrJVh7T2cqIFfMR"
consumer_secret = "yah2rFbZE1qX9n1MDaqObT1PfCOSYlvxIyXYuCO4ZrSUfSvjEC"
access_key = "781555439183859712-tS4fKwo03ujDtNL0UibVIXauXRFdFvs"
access_secret = "3YrATCHVvP2atvFs4M6VJ0dFvsdE3BBKoZ2OcvKdqPzJp"

# OAuth
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)

# API Call
api = tweepy.API(auth, parser = tweepy.parsers.JSONParser())

In [2]:
# Test API by pulling data from Donald Trumps twitter
data = api.user_timeline(id='realDonaldTrump', count = 200)  #Example:@realDonaldTrump
for i in range(10):
    print([i],data[i]['text'])

[0] It was an honor to welcome the Prime Minister of Vietnam, Nguyễn Xuân Phúc to the @WhiteHouse this afternoon. https://t.co/6YbrIvJUCY
[1] We traveled the world to strengthen long-standing alliances, and to form new partnerships. See more at:… https://t.co/FT4kyPoAl0
[2] I will be announcing my decision on the Paris Accord over the next few days. MAKE AMERICA GREAT AGAIN!
[3] Hopefully Republican Senators, good people all, can quickly get together and pass a new (repeal &amp; replace) HEALTHCARE bill. Add saved $'s.
[4] Kathy Griffin should be ashamed of herself. My children, especially my 11 year old son, Barron, are having a hard time with this. Sick!
[5] ...case against him &amp; now wants to clear his name by showing "the false or misleading testimony by James Comey, John Brennan..." Witch Hunt!
[6] So now it is reported that the Democrats, who have excoriated Carter Page about Russia, don't want him to testify. He blows away their....
[7] Who can figure out the true meaning of 

In [3]:
search_query =  '"Ethereum" -filter:retweets'
data = api.search(q = search_query, count = 100, lang = 'en', result_type = 'mixed')
all_data = list(data.values())[1]

all_data[1]['text']

'Bitcoin could be eclipsed by ethereum https://t.co/SPSMLKMipq https://t.co/k11IRz2f0M'

In [4]:
while(len(all_data) <= 1000):
    #time.sleep(5)
    last = all_data[-1]['id']
    data = api.search(q = search_query, count = 100, lang = 'en', result_type = 'mixed', since_id = last)
    all_data += list(data.values())[1][1:]


In [5]:
len(all_data)

1013

In [8]:
tweet = []
number_favourites = []
vs_compound = []
vs_pos = []
vs_neu = []
vs_neg = []

analyzer = SentimentIntensityAnalyzer()

for i in range(0, len(all_data)):
    tweet.append(all_data[i]['text'])
    number_favourites.append(all_data[i]['favorite_count'])
    vs_compound.append(analyzer.polarity_scores(all_data[i]['text'])['compound'])
    vs_pos.append(analyzer.polarity_scores(all_data[i]['text'])['pos'])
    vs_neu.append(analyzer.polarity_scores(all_data[i]['text'])['neu'])
    vs_neg.append(analyzer.polarity_scores(all_data[i]['text'])['neg'])

In [12]:
twitter_df = DataFrame({'Tweet': tweet,
                        'Favourites': number_favourites,
                        'Compound': vs_compound,
                        'Positive': vs_pos,
                        'Neutral': vs_neu,
                        'Negative': vs_neg})

twitter_df = twitter_df[['Tweet', 'Favourites', 'Compound',
                         'Positive', 'Neutral', 'Negative']]

# Have a look at the top 5 results.
twitter_df.head()

,Tweet,Favourites,Compound,Positive,Neutral,Negative
0,https://t.co/oiFGyh1iju This is a $2220 tx fee...,289,0.4404,0.185,0.738,0.077
1,Bitcoin could be eclipsed by ethereum https://...,59,0.0000,0.000,1.000,0.000
2,Ethereum (Ether) Could Be Worth More Than Bitc...,118,0.2263,0.160,0.840,0.000
3,My ethereum and litecoin purchase finally clea...,0,0.2462,0.221,0.779,0.000
4,zerohedge: Ethereum Forecast To Surpass Bitcoi...,1,0.0000,0.000,1.000,0.000
